In [1]:
# 0 := delete \n
# 1 := leave as it is

# model = gensim.models.Word2Vec.load('../../../pretrained_model/kor/ko.bin')

# from hangul_utils import split_syllables, join_jamos
# import gensim

In [2]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init
import tester
import model_util as mu

In [3]:
first_np, second_np, label_np  = data.getData()
first_np, second_np = utils.process_splitted(first_np, second_np)
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np)
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

linearized complete!
done tokenizing both data!
saved ch2idx to file!
encoding comlete!


In [4]:
pretrained_word2vec = pm.load_pretrained_model(ch2idx)
pretrained_word2vec = torch.tensor(pretrained_word2vec)

There are 27 / 133 pretrained vectors found.


In [5]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

In [6]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=50)

In [7]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('rnn-ocr/tests')

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [8]:
# ====================
# set parameters here
# ====================

title = 'rnn-test11'
epochs = 40

vocab_size=len(ch2idx)
embed_dim = 200
hidden_size = 100
num_classes = 2
rnn_layers = 1

num_filters = [100, 200, 100]
kernel_sizes = [15, 21, 14]

dropout = 0.5

learning_rate = 0.001
weight_decay = 1e-4

model_name="RNN"
optim_name="Adam"
loss_fn_name="CEL"

pretrained_model=pretrained_word2vec
freeze_embedding=False,

In [9]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing with pretrained model!!!


/home/arise/anaconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


OCR_rnn(
  (emb): Embedding(133, 200)
  (lstm1): RNN(
    (rnn): LSTM(200, 100, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(200, 100, batch_first=True, dropout=0.5, bidirectional=True)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.5, inplace=False)
  (dp2): Dropout(p=0.5, inplace=False)
)


In [10]:
trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.286497   | 89.111111  | 0.190453 | 93.67  | 37.98 
   2    |   0.198404   | 93.156379  | 0.155225 | 94.96  | 38.14 
   3    |   0.168802   | 94.102881  | 0.133788 | 95.15  | 37.95 
   4    |   0.152975   | 94.283951  | 0.128070 | 95.74  | 37.94 
   5    |   0.145939   | 94.526749  | 0.130579 | 95.37  | 38.12 
   6    |   0.139404   | 94.781893  | 0.122297 | 95.44  | 38.21 
   7    |   0.134683   | 95.090535  | 0.120333 | 95.22  | 38.04 
   8    |   0.131919   | 95.148148  | 0.127021 | 95.52  | 38.27 
   9    |   0.129080   | 95.275720  | 0.120378 | 95.33  | 38.33 
  10    |   0.123248   | 95.469136  | 0.121046 | 95.67  | 38.24 
  11    |   0.120397   | 95.576132  | 0.117072 | 95.78  | 37.95 
  12    |   0.118503   | 95.740741  | 0.117028 | 95.93  | 38.11 
  13    |   0.114057   | 95.724280  | 0.129221 | 95.6

In [11]:
tester.test(test_dataloader=test_dataloader,
            device=device,
            model=model,
            title=title)

test loss:  0.1594668993493542
test acc:  95.39999999999999
saved precision and recall results to file!


(tensor([0., 1., 1.,  ..., 0., 0., 0.], device='cuda:0'),
 tensor([0., 1., 1.,  ..., 0., 0., 0.], device='cuda:0'))

In [12]:
# mu.graphModel(train_dataloader, model, writer)

In [13]:
mu.saveModel(title, model)